In [ ]:
import pandas as pd
import os

os.chdir(r'C:\Users\jf79\Repos\REAP-Analytics')
import myfuncs.myfuncs as mf


import warnings
warnings.filterwarnings('ignore')

path_to_data = r"C:\Users\jf79\OneDrive - Office Shared Service\Sharepoint Links\BI - Corporate - Documents\REAP\1. REAP Monthly Report 2025\Netcall\All Historical Data\Interactions Data"
os.chdir(path_to_data)
mf.read_directory()

master_data = pd.read_csv('Netcall Interactions Master Data.csv',low_memory=False)
master_data = mf.strip_dataframe(master_data)


In [2]:
master_df = master_data.copy()

columns_to_drop = [
    'customername','assistedby',
    'casereference','ipaddress'
]
master_df.drop(columns=columns_to_drop, inplace=True)

master_df['nullcount'] = master_df.isna().sum(axis=1)
master_df.sort_values(by=['interactionid','nullcount'], inplace=True)
drop = master_df[master_df['interactionid'].duplicated(keep='first')]
master_df.drop(drop.index, inplace=True)
master_df.drop(columns='nullcount', inplace=True)

columns_to_change = ['timeended', 'timequeued']
for column in columns_to_change:
    master_df[column] = pd.to_datetime(master_df[column])

master_df['queuetime'] = master_df['timeended'] - master_df['timequeued']

columns_to_change = [
    'duration','talktime','holdtime',
    'wrapupduration','agenthandlingtime'
]
master_df[columns_to_change] = master_df[columns_to_change].apply(lambda x: x.str.replace(r'(?<!^)-', '', regex=True))
for column in columns_to_change:
    master_df[column] = pd.to_timedelta(master_df[column])

master_df['dateofqueue'] = master_df['timequeued'].dt.date

mf.validate_data(master_df)

In [ ]:
aggregated_interactions = master_df.groupby(['direction','type','dateofqueue','groupname','queue','result']).agg(
    number_of_interactions = ('interactionid', 'count'),
    wrap_upduration = ('wrapupduration','sum'),
    queue_time = ('queuetime','sum'),
    duration = ('duration','sum'),
    talk_time = ('talktime','sum'),
    hold_time = ('holdtime','sum'),
    agent_handling_time = ('agenthandlingtime','sum'),
).reset_index()

columns_to_change = [
    'wrap_upduration','queue_time','duration',
    'talk_time','hold_time','agent_handling_time'
]
for column in columns_to_change:
    aggregated_interactions[column].fillna(pd.Timedelta(0), inplace=True)
    aggregated_interactions[column] = aggregated_interactions[column].apply(lambda x: x.total_seconds())

mf.validate_data(aggregated_interactions)

In [ ]:
mf.export_file(
    aggregated_interactions,
    directory = path_to_data,
    df_name='Netcall Dashboard Data'
)